In [1]:
import csv
import joblib,pickle
from numpy import array,transpose
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt 
from uncertainties import ufloat, unumpy


jet_pt=['500-600', '600-800', '800-1000','1000-1200','1200-1500','1500-2000']


In [2]:
nominal_path = '/global/cfs/projectdirs/atlas/wys/QG_Calibration/uncertainty/SFs_pkls'
nominal_path = Path(nominal_path)

reweighting_vars = ['jet_nTracks', 'GBDT_newScore']
nominal_keys = [reweighting_var + '_quark_reweighting_weights' for reweighting_var in reweighting_vars]
WPs = [0.5, 0.6, 0.7, 0.8]
partons=['Quark','Gluon']

In [3]:
syst_total = joblib.load('/global/cfs/projectdirs/atlas/wys/QG_Calibration/uncertainty/syst_total.pkl')

In [4]:
stats_path = joblib.load('/global/cfs/projectdirs/atlas/wys/QG_Calibration/uncertainty/SFs_bootstrap')

In [6]:
stats_path['jet_nTracks'][0.5]['Quark']

array([1.027949246082951+/-0.0014749402917943793,
       1.0443619951271408+/-0.0012214622180948898,
       1.0461863927945312+/-0.0020971006907254857,
       1.0334177625950154+/-0.004020719813153815,
       1.0204479508869104+/-0.007235909067873102,
       1.0096590131026066+/-0.015376481531858027], dtype=object)

In [5]:

unc_total=syst_total

for var,revar in unc_total.items():
     if 'statistical' not in unc_total[var]:
          unc_total[var]['statistical']={} 
     
     for syst, syst_dict in revar.items():
          for wp,wp_dict in syst_dict.items():
              
               if wp not in unc_total[var]['statistical']:
                    unc_total[var]['statistical'][wp]={}

               for par,par_dict in wp_dict.items():
                    if par not in unc_total[var]['statistical'][wp]:
                         unc_total[var]['statistical'][wp][par]={}
                              
                    stats = unumpy.std_devs(stats_path[var][wp][par])
                    #print(stats)
                    unc_total[var]['statistical'][wp][par]=stats


unc_total['jet_nTracks']

{'trk_eff': {0.5: {'Quark': array([0.07838215, 0.07105041, 0.06370761, 0.05383175, 0.04803941,
          0.0399023 ]),
   'Gluon': array([0.0148738 , 0.0165012 , 0.01774697, 0.01621967, 0.01536355,
          0.0123203 ])},
  0.6: {'Quark': array([0.07708517, 0.06749625, 0.05893219, 0.0517522 , 0.0453059 ,
          0.03591566]),
   'Gluon': array([0.01551961, 0.01638574, 0.01658754, 0.01688472, 0.01609701,
          0.00952872])},
  0.7: {'Quark': array([0.07454819, 0.06302027, 0.05462789, 0.0473304 , 0.03943486,
          0.03287973]),
   'Gluon': array([0.01548032, 0.01524581, 0.01551349, 0.01537917, 0.01325577,
          0.01323624])},
  0.8: {'Quark': array([0.06427668, 0.05343191, 0.04594418, 0.03787851, 0.03049519,
          0.02480305]),
   'Gluon': array([0.01061766, 0.01012699, 0.01076818, 0.01094601, 0.00763662,
          0.00500626])}},
 'JESJER': {0.5: {'Quark': array([0.00313189, 0.00287976, 0.00285112, 0.00262912, 0.00323266,
          0.0022695 ]),
   'Gluon': array([0.0

In [6]:
nominal_SFs={}
for i_var, reweighting_var in enumerate(reweighting_vars):

    nominal_SFs[reweighting_var] = joblib.load(nominal_path / nominal_keys[i_var] / "SFs.pkl") # this includes many vars with WPs 
    
#nominal_SFs1=joblib.load('/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/NewWorkflow/trained_lightGBM_new/nominal/plots/ADE/SFs_pkls/jet_nTracks_quark_reweighting_weights/SFs.pkl')
#nominal_SFs2=joblib.load('/global/cfs/projectdirs/atlas/wys/HEP_Repo/QG_Calibration/NewWorkflow/trained_lightGBM_new/nominal/plots/ADE/SFs_pkls/jet_nTracks_quark_reweighting_weights/SFs.pkl')
nominal_SFs['jet_nTracks']['jet_nTracks'] # 1st is reweighting var, 2nd is tagger

{0.5: {'Quark': array([1.0279333381553586+/-0.0018586113195646565,
         1.0443329347783434+/-0.0015922765556675275,
         1.0461937516008866+/-0.0028960158934784102,
         1.033476695908774+/-0.005694787038356356,
         1.0204495764454495+/-0.010449997703414317,
         1.00974996065689+/-0.02274082580019074], dtype=object),
  'Gluon': array([0.9976396644241626+/-0.0015426248605250306,
         0.9918263566430869+/-0.0015110416048499872,
         0.9835812581791289+/-0.003159745227864179,
         0.9650002946311069+/-0.0070948285520520075,
         0.944176894562725+/-0.014411382245602048,
         0.9246181658202915+/-0.0404637170522422], dtype=object)},
 0.6: {'Quark': array([1.0328955737461314+/-0.001750771722367234,
         1.044722955032124+/-0.0014986834173378503,
         1.0465884898001971+/-0.0027266553307359557,
         1.0335629015160124+/-0.005372809554784582,
         1.017783118505041+/-0.009751816323492205,
         1.01783619174369+/-0.02139744215277465

In [9]:
nom={}
new_data={}
sorted_data={}
convert={}
for var, revar in unc_total.items():
    new_data[var]={}
    #print(revar.keys())
    if 'trk_eff' in revar:
        revar['tracking']=revar.pop('trk_eff')
    if 'JESJER' in revar:
        revar['JES/JER']=revar.pop('JESJER')
        #del revar['trk_eff']
    for syst, trk_dict in revar.items():
        syst=syst.replace('_', ' ')
        #print(syst)
        for wp, values in trk_dict.items():
            for jet_type, arr in values.items():
                ### re-order the keys
                if wp not in new_data[var]:
                    new_data[var][wp] = {}
                if jet_type not in new_data[var][wp]:
                    new_data[var][wp][jet_type] = {}   
                
                arr2 = abs(np.round(arr,4))
                new_data[var][wp][jet_type][syst]=arr2
                    #new_data[var][wp][jet_type]['nominal']=
            
                    # sorted_data={k1:{k2:{k3: sorted(v3.items(),key=lambda x:x[1][0],reverse=True)
                    #                                    for k3,v3 in v2.items()}
                    #                                    for k2,v2 in v1.items()}
                    #                                   for k1,v1 in new_data.items()}
# #### rank the values
# for var, revar in sorted_data.items():   
#     for wp, values in revar.items():
#         for jet_type, arr in values.items():
#             for s,sv in arr:
#                 if var not in convert:
#                     convert[var]={}
#                 if wp not in convert[var]:
#                     convert[var][wp]={}         
#                 if jet_type not in convert[var][wp]:
#                     convert[var][wp][jet_type]={}
                            
#                         #convert[var][wp][jet_type]=dict.fromkeys(s)
#                 convert[var][wp][jet_type][s]=sv
                    
# print(convert)

In [11]:
nominal_SFs1={}

for v in ['jet_nTracks', 'GBDT_newScore']:
    for w in [0.5,0.6,0.7,0.8]:
        wp = w*100
        SF_arr={}
        for j in ['Quark','Gluon']:
            new_data1 = new_data[v][w][j]
  
            df = pd.DataFrame.from_dict(new_data1, orient="index",columns=jet_pt)

            df2 = df.round(decimals=2)
            df2.replace(0.00,0.01,inplace=True)
            df2.loc['Total Uncertainty']=np.sqrt(np.sum(np.power(df.iloc[1:],2)))
            ##syst only
            df2.to_latex(f'./tables/syst_{v}_{w}_{j}.tex',caption=f'Uncertainties of {v} at {wp}\% WP for {j}',label=f'tab:syst_{v}_{w}_{j}',float_format="%.2f",column_format='ccccccc')

            arr_nom = unumpy.nominal_values(nominal_SFs[v][v][w][j])
            nominal_SFs1[j]=arr_nom

            ## for SF with syst
            SF_arr1=unumpy.uarray(arr_nom,df2.loc['Total Uncertainty'])
            SF_arr[j]=SF_arr1

        SF_pd=pd.DataFrame.from_dict(SF_arr,orient="index",columns=jet_pt)
        SF_pd.to_latex(float_format="%.2f",column_format='ccccccc') 
        SF_pd.to_latex(f'./tables/SF_syst_{v}_{w}.tex',caption=f'Scale Factors of {v} at {wp}\% WP',label=f'tab:{v}_{w}',float_format="%.2f",column_format='ccccccc') 
        #print(SF_pd)


        # SF only
        df_nom=pd.DataFrame.from_dict(nominal_SFs1, orient="index",columns=jet_pt)

        df_nom.to_latex(f'./tables/SF_{v}_{w}.tex',caption=f'Scale Factors of {v} at {wp}\% WP',label=f'tab:{v}_{w}_{j}',float_format="%.2f",column_format='ccccccc')



/tmp/ipykernel_39858/2021054523.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df2.to_latex(f'./tables/syst_{v}_{w}_{j}.tex',caption=f'Uncertainties of {v} at {wp}\% WP for {j}',label=f'tab:syst_{v}_{w}_{j}',float_format="%.2f",column_format='ccccccc')
/tmp/ipykernel_39858/2021054523.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df2.to_latex(f'./tables/syst_{v}_{w}_{j}.tex',caption=f'Uncertainties of {v} at {wp}\% WP for {j}',label=f'tab:syst_{v}_{w}_{j}',float

           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.03+/-0.19  1.04+/-0.19  1.05+/-0.21  1.03+/-0.20  1.02+/-0.21   
Gluon  1.00+/-0.11  0.99+/-0.12  0.98+/-0.13  0.97+/-0.15  0.94+/-0.14   

         1500-2000  
Quark  1.01+/-0.19  
Gluon  0.92+/-0.17  
           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.03+/-0.16  1.04+/-0.16  1.05+/-0.18  1.03+/-0.18  1.02+/-0.18   
Gluon  1.00+/-0.17  0.99+/-0.18  0.98+/-0.18  0.96+/-0.19  0.93+/-0.19   

         1500-2000  
Quark  1.02+/-0.17  
Gluon  0.92+/-0.24  
           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.04+/-0.14  1.04+/-0.14  1.04+/-0.15  1.03+/-0.15  1.02+/-0.15   
Gluon  0.99+/-0.23  0.98+/-0.23  0.97+/-0.24  0.95+/-0.26  0.92+/-0.27   

         1500-2000  
Quark  1.01+/-0.13  
Gluon  0.87+/-0.33  
           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.03+/-0.11  1.04+/-0.11  1.04+/-0.12  1.03+/-0.11  1.02+/-0.

/tmp/ipykernel_39858/2021054523.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df2.to_latex(f'./tables/syst_{v}_{w}_{j}.tex',caption=f'Uncertainties of {v} at {wp}\% WP for {j}',label=f'tab:syst_{v}_{w}_{j}',float_format="%.2f",column_format='ccccccc')
/tmp/ipykernel_39858/2021054523.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  SF_pd.to_latex(float_format="%.2f",column_format='ccccccc')
/tmp/ipykernel_39858/2021054523.py:34: FutureWarning: In future versions `

           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.02+/-0.15  1.03+/-0.16  1.04+/-0.17  1.03+/-0.18  1.02+/-0.18   
Gluon  0.99+/-0.13  0.98+/-0.13  0.97+/-0.15  0.95+/-0.15  0.93+/-0.16   

         1500-2000  
Quark  1.01+/-0.15  
Gluon  0.91+/-0.21  
           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.03+/-0.13  1.04+/-0.13  1.04+/-0.15  1.03+/-0.15  1.02+/-0.15   
Gluon  0.98+/-0.18  0.97+/-0.17  0.95+/-0.19  0.93+/-0.19  0.89+/-0.20   

         1500-2000  
Quark  1.02+/-0.12  
Gluon  0.86+/-0.25  
           500-600      600-800     800-1000    1000-1200    1200-1500  \
Quark  1.03+/-0.11  1.04+/-0.11  1.04+/-0.12  1.03+/-0.12  1.02+/-0.11   
Gluon  0.96+/-0.22  0.95+/-0.21  0.93+/-0.23  0.90+/-0.23  0.85+/-0.26   

         1500-2000  
Quark  1.01+/-0.10  
Gluon  0.81+/-0.32  


/tmp/ipykernel_39858/2021054523.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_nom.to_latex(f'./tables/SF_{v}_{w}.tex',caption=f'Scale Factors of {v} at {wp}\% WP',label=f'tab:{v}_{w}_{j}',float_format="%.2f",column_format='ccccccc')


In [213]:
df2.loc['Total Uncertainty']

500-600      0.221847
600-800      0.213256
800-1000     0.230781
1000-1200    0.232905
1200-1500    0.257137
1500-2000    0.320260
Name: Total Uncertainty, dtype: float64

In [215]:
df_nom.loc['Quark']

500-600      1.029889
600-800      1.038596
800-1000     1.041077
1000-1200    1.033131
1200-1500    1.022275
1500-2000    1.012740
Name: Quark, dtype: float64

In [230]:
SF_arr=[]
SF_arr1=unumpy.uarray(df_nom.loc['Quark'],df2.loc['Total Uncertainty'])
SF_arr2=unumpy.uarray(df_nom.loc['Gluon'],df2.loc['Total Uncertainty'])
SF_arr.append(SF_arr1)
SF_arr.append(SF_arr2)

SF_arr

[array([1.0298886352036967+/-0.2218465911390121,
        1.0385964951324744+/-0.21325564470841094,
        1.0410768657219311+/-0.2307813467332228,
        1.0331313264399509+/-0.23290510943300494,
        1.0222748356715425+/-0.2571371812865654,
        1.0127399433179811+/-0.3202598788484127], dtype=object),
 array([0.9576568443022633+/-0.2218465911390121,
        0.9452818632438846+/-0.21325564470841094,
        0.929125507211347+/-0.2307813467332228,
        0.9024682319456646+/-0.23290510943300494,
        0.8516554416032521+/-0.2571371812865654,
        0.8072196189660318+/-0.3202598788484127], dtype=object)]

In [232]:
SF_pd=pd.DataFrame(SF_arr,columns=jet_pt)
SF_pd

,500-600,600-800,800-1000,1000-1200,1200-1500,1500-2000
0,1.03+/-0.22,1.04+/-0.21,1.04+/-0.23,1.03+/-0.23,1.02+/-0.26,1.01+/-0.32
1,0.96+/-0.22,0.95+/-0.21,0.93+/-0.23,0.90+/-0.23,0.85+/-0.26,0.81+/-0.32


In [233]:
SF_pd.to_latex()
SF_pd

/tmp/ipykernel_234529/4264593941.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  SF_pd.to_latex()


,500-600,600-800,800-1000,1000-1200,1200-1500,1500-2000
0,1.03+/-0.22,1.04+/-0.21,1.04+/-0.23,1.03+/-0.23,1.02+/-0.26,1.01+/-0.32
1,0.96+/-0.22,0.95+/-0.21,0.93+/-0.23,0.90+/-0.23,0.85+/-0.26,0.81+/-0.32
